<a href="https://colab.research.google.com/github/springboardmentor458/OBJECT_RECOGNITION_SYSTEM_Infosys_Internship_Oct2024/blob/Harsha-Kota/springboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless albumentations torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
import torch
import os
import json
import urllib.request

Use this if you need all those images in folder

In [ ]:
#Remove comments to run
"""
# Cell 1
import json
import cv2
import urllib.request
import numpy as np
import os

json_filename = '/content/drive/MyDrive/Colab Notebooks/instances_val2017.json'
with open(json_filename, 'r') as f:
    data = json.load(f)

output_dir = '/content/output_images'
os.makedirs(output_dir, exist_ok=True)

def download_and_save(image_url, annotations, filename):
    resp = urllib.request.urlopen(image_url)
    img_array = np.asarray(bytearray(resp.read()), dtype=np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    for ann in annotations:
        x, y, w, h = ann['bbox']
        cv2.rectangle(img, (int(x), int(y)), (int(x+w), int(y+h)), (255, 0, 0), 2)
    output_path = os.path.join(output_dir, filename)
    cv2.imwrite(output_path, img)

for img_data in data['images']:
    img_id = img_data['id']
    img_url = img_data['coco_url']
    filename = img_data['file_name']
    annotations = [ann for ann in data.get('annotations', []) if ann['image_id'] == img_id]
    download_and_save(img_url, annotations, filename)

print(f"All images saved in {output_dir}")
"""

Use This for Direct Representations

In [ ]:
# Cell 1
import json
import cv2
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

json_filename = '/content/drive/MyDrive/Colab Notebooks/instances_val2017.json'
with open(json_filename, 'r') as f:
    data = json.load(f)

def download_and_display(image_url, annotations, image=None):
    resp = urllib.request.urlopen(image_url)
    img_array = np.asarray(bytearray(resp.read()), dtype=np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

    if image is not None:
        img = image

    for ann in annotations:
        x, y, w, h = ann['bbox']
        cv2.rectangle(img, (int(x), int(y)), (int(x+w), int(y+h)), (255, 0, 0), 2)

    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# Get the user input for the number of images to load
num_images = int(input("Enter the number of images to load (default is all): ") or len(data['images']))

original_images = []
original_annotations = []

for i, img_data in enumerate(data['images']):
    if i >= num_images:
        break
    img_id = img_data['id']
    img_url = img_data['coco_url']
    annotations = [ann for ann in data.get('annotations', []) if ann['image_id'] == img_id]
    original_images.append(img_url)
    original_annotations.append(annotations)
    download_and_display(img_url, annotations)

Image Processing Pipeline with Annotations and Visualization(Test 1)

In [ ]:
# Cell 2
import cv2
import numpy as np
from PIL import Image

def process_images(original_images, original_annotations):
    processed_images = []
    processed_annotations = []

    for img_url, annotations in zip(original_images, original_annotations):
        resp = urllib.request.urlopen(img_url)
        img_array = np.asarray(bytearray(resp.read()), dtype=np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

        # Filter images
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        binary_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

        # Resize images
        resized_img = cv2.resize(img, (224, 224))

        # Test for correct orientation
        height, width, _ = img.shape
        if height > width:
            rotated_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        else:
            rotated_img = img

        # Fix blurry images
        sharpened_img = cv2.GaussianBlur(rotated_img, (0, 0), sigmaX=1, sigmaY=1)
        sharpened_img = cv2.addWeighted(rotated_img, 1.5, sharpened_img, -0.5, 0)

        # Apply filters (e.g., black and white)
        bw_img = cv2.cvtColor(sharpened_img, cv2.COLOR_BGR2GRAY)

        processed_images.extend([img, binary_img, resized_img, rotated_img, sharpened_img, bw_img])
        processed_annotations.extend([annotations] * 6)

    return processed_images, processed_annotations

# Call the processing function with the data from the previous cell
processed_images, processed_annotations = process_images(original_images, original_annotations)

# Display the processed images
for i, (img, annotations) in enumerate(zip(processed_images, processed_annotations)):
    download_and_display(original_images[i % len(original_images)], annotations, image=img)

Image Processing Pipeline with Annotations and Visualization(Test 2)

In [ ]:
# Cell 2
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

def process_and_display_images(original_images, original_annotations):
    def draw_annotations(image, annotations):
        # Create a copy of the image to avoid modifying the original
        annotated_img = image.copy()
        for ann in annotations:
            x, y, w, h = ann['bbox']
            cv2.rectangle(annotated_img,
                         (int(x), int(y)),
                         (int(x+w), int(y+h)),
                         (255, 0, 0), 2)
        return annotated_img

    def create_image_grid(images, titles, num_cols=3):
        num_images = len(images)
        num_rows = (num_images + num_cols - 1) // num_cols

        fig = plt.figure(figsize=(15, 5 * num_rows))
        gs = GridSpec(num_rows, num_cols, figure=fig)

        for idx, (img, title) in enumerate(zip(images, titles)):
            row = idx // num_cols
            col = idx % num_cols
            ax = fig.add_subplot(gs[row, col])

            if len(img.shape) == 2:  # If grayscale
                # Convert grayscale to BGR for annotation
                img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
                ax.imshow(cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB))
            else:  # If color
                ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

            ax.set_title(title)
            ax.axis('off')

        plt.tight_layout()
        plt.show()

    for img_url, annotations in zip(original_images, original_annotations):
        # Download and process each image
        resp = urllib.request.urlopen(img_url)
        img_array = np.asarray(bytearray(resp.read()), dtype=np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

        # Process images with different techniques
        processed_images = []
        titles = []

        # 1. Original Image with annotations
        annotated_original = draw_annotations(img, annotations)
        processed_images.append(annotated_original)
        titles.append('Original Image with Annotations')

        # 2. Binary Image with annotations
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        binary_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        binary_img_color = cv2.cvtColor(binary_img, cv2.COLOR_GRAY2BGR)
        annotated_binary = draw_annotations(binary_img_color, annotations)
        processed_images.append(annotated_binary)
        titles.append('Binary Image with Annotations')

        # 3. Resized Image with annotations
        resized_img = cv2.resize(img, (224, 224))
        # Scale annotations for resized image
        scale_x = 224.0 / img.shape[1]
        scale_y = 224.0 / img.shape[0]
        scaled_annotations = []
        for ann in annotations:
            scaled_ann = ann.copy()
            x, y, w, h = ann['bbox']
            scaled_ann['bbox'] = [x * scale_x, y * scale_y, w * scale_x, h * scale_y]
            scaled_annotations.append(scaled_ann)
        annotated_resized = draw_annotations(resized_img, scaled_annotations)
        processed_images.append(annotated_resized)
        titles.append('Resized Image (224x224) with Annotations')

        # 4. Rotated Image with annotations (portrait orientation images will rotate.)
        height, width, _ = img.shape
        if height > width:              # If you want to rotate all images regardless of their orientation remove the line or adjust
            rotated_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
            # Adjust annotations for rotation
            rotated_annotations = []
            for ann in annotations:
                rotated_ann = ann.copy()
                x, y, w, h = ann['bbox']
                rotated_ann['bbox'] = [y, width - x - w, h, w]
                rotated_annotations.append(rotated_ann)
            rotation_status = "Rotated 90°"
            annotated_rotated = draw_annotations(rotated_img, rotated_annotations)
        else:
            rotated_img = img
            rotation_status = "No Rotation Needed"
            annotated_rotated = draw_annotations(rotated_img, annotations)
        processed_images.append(annotated_rotated)
        titles.append(f'Orientation Check\n({rotation_status})')

        # 5. Sharpened Image with annotations
        sharpened_img = cv2.GaussianBlur(rotated_img, (0, 0), sigmaX=1, sigmaY=1)
        sharpened_img = cv2.addWeighted(rotated_img, 1.5, sharpened_img, -0.5, 0)
        annotated_sharpened = draw_annotations(sharpened_img,
                                             rotated_annotations if height > width else annotations)
        processed_images.append(annotated_sharpened)
        titles.append('Sharpened Image with Annotations')

        # 6. Black and White Image with annotations
        bw_img = cv2.cvtColor(sharpened_img, cv2.COLOR_BGR2GRAY)
        bw_img_color = cv2.cvtColor(bw_img, cv2.COLOR_GRAY2BGR)
        annotated_bw = draw_annotations(bw_img_color,
                                      rotated_annotations if height > width else annotations)
        processed_images.append(annotated_bw)
        titles.append('Black & White Image with Annotations')

        # Display grid for current image
        create_image_grid(processed_images, titles)

# Usage:
processed_images, processed_annotations = process_and_display_images(original_images, original_annotations)

Custom PyTorch Dataset Class for COCO-style Object Detection

In [ ]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import json
import albumentations as A
from albumentations.pytorch import ToTensorV2
import multiprocessing

class CocoDataset(Dataset):
    def __init__(self, images, annotations, category_mapping, img_dir, transform=None):
        self.images = images
        self.annotations = annotations
        self.category_mapping = category_mapping
        self.img_dir = img_dir
        self.transform = transform
        self.image_id_to_annotations = self._group_annotations_by_image()
        print("COCO Dataset Class initialized successfully!")

    def _group_annotations_by_image(self):
        image_id_to_annotations = {}
        for ann in self.annotations:
            image_id = ann['image_id']
            if image_id not in image_id_to_annotations:
                image_id_to_annotations[image_id] = []
            image_id_to_annotations[image_id].append(ann)
        return image_id_to_annotations

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_info = self.images[idx]
        img_path = os.path.join(self.img_dir, image_info['file_name'])

        # Load image
        try:
            image = cv2.imread(img_path)
            if image is not None:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                print(f"Successfully loaded image: {img_path}")
            else:
                raise FileNotFoundError(f"Image not found or failed to load: {img_path}")
        except Exception as e:
            print(f"Error loading image at {img_path}: {e}")
            return None, None

        # Get annotations
        image_id = image_info['id']
        annotations = self.image_id_to_annotations.get(image_id, [])
        if not annotations:
            print(f"No annotations found for image ID {image_id}")
            return None, None

        boxes = []
        labels = []
        for ann in annotations:
            x, y, width, height = ann['bbox']
            boxes.append([x, y, x + width, y + height])
            labels.append(self.category_mapping.get(ann['category_id'], -1))  # Default to -1 if not found

        # Convert boxes and labels to numpy arrays
        try:
            boxes = np.array(boxes, dtype=np.float32)
            labels = np.array(labels, dtype=np.int64)
        except ValueError as e:
            print(f"Error processing labels for image ID {image_id}: {e}")
            return None, None

        # Apply transformations
        if self.transform:
            transformed = self.transform(image=image, bboxes=boxes, labels=labels)
            image = transformed['image']
            boxes = transformed['bboxes']
            labels = transformed['labels']
        else:
            print(f"No transformations applied to image ID {image_id}")

        # Convert to PyTorch tensors
        if len(boxes) == 0 or len(labels) == 0:
            print(f"Warning: No valid boxes or labels for image ID {image_id}")
            return None, None

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)
        target = {"boxes": boxes, "labels": labels}

        return image, target


# Load JSON file
with open('/content/drive/MyDrive/Colab Notebooks/instances_val2017.json', 'r') as f:
    try:
        coco_data = json.load(f)
        print("JSON file loaded successfully!")
    except json.JSONDecodeError as e:
        print(f"Error loading JSON file: {e}")
        coco_data = {}

images = coco_data.get('images', [])
annotations = coco_data.get('annotations', [])
categories = coco_data.get('categories', [])

if not images or not annotations or not categories:
    print("Warning: Missing data in COCO JSON.")
else:
    print(f"Loaded {len(images)} images, {len(annotations)} annotations, and {len(categories)} categories.")

# Map category IDs to numeric labels
category_mapping = {category['id']: idx for idx, category in enumerate(categories)}

# Augmentation and Preprocessing Pipeline
transform = A.Compose([
    A.Resize(height=416, width=416),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.2),
    A.GaussianBlur(blur_limit=(3, 7), p=0.2),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=20, p=0.5),
    A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1, p=0.3),
    A.ToGray(p=0.1),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

coco_dataset = CocoDataset(images, annotations, category_mapping, '/content/drive/MyDrive/Colab Notebooks/val2017', transform=transform)

# Adjust num_workers based on CPU count
num_workers = min(2, multiprocessing.cpu_count())
data_loader = DataLoader(coco_dataset, batch_size=16, shuffle=True, collate_fn=lambda x: tuple(zip(*x)), num_workers=num_workers)

# Process DataLoader
num_valid_samples = 0
num_skipped_samples = 0
for images, targets in data_loader:
    if images is not None and targets is not None:
        print(f"Images batch size: {len(images)}")
        print(f"Target batch size: {len(targets)}")
        print("Sample target:", targets[0])
        num_valid_samples += len(images)
    else:
        num_skipped_samples += 1
        print(f"Skipping invalid sample ({num_skipped_samples})")

print(f"Total number of valid samples: {num_valid_samples}")
print(f"Total number of skipped samples: {num_skipped_samples}")


In [ ]:
#improved
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import json
import albumentations as A
from albumentations.pytorch import ToTensorV2
import multiprocessing
from tqdm import tqdm


class CocoDataset(Dataset):
    def __init__(self, images, annotations, category_mapping, img_dir, transform=None, input_size=(416, 416)):
        self.images = images
        self.annotations = annotations
        self.category_mapping = category_mapping
        self.img_dir = img_dir
        self.transform = transform
        self.input_size = input_size
        self.image_id_to_annotations = self._group_annotations_by_image()
        print(f"COCO Dataset initialized with {len(images)} images.")

    def _group_annotations_by_image(self):
        image_id_to_annotations = {}
        for ann in self.annotations:
            image_id = ann['image_id']
            if image_id not in image_id_to_annotations:
                image_id_to_annotations[image_id] = []
            image_id_to_annotations[image_id].append(ann)
        return image_id_to_annotations

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_info = self.images[idx]
        img_path = os.path.join(self.img_dir, image_info['file_name'])

        # Load and preprocess image
        try:
            image = cv2.imread(img_path)
            if image is None:
                raise FileNotFoundError(f"Image not found: {img_path}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            return None, None

        # Get original image size
        original_size = image.shape[:2]

        # Resize image dynamically to the input size
        image = cv2.resize(image, (self.input_size[1], self.input_size[0]))

        # Get annotations for the image
        image_id = image_info['id']
        annotations = self.image_id_to_annotations.get(image_id, [])
        if not annotations:
            return None, None

        boxes = []
        labels = []
        for ann in annotations:
            x, y, width, height = ann['bbox']
            # Convert bbox to format (xmin, ymin, xmax, ymax)
            boxes.append([x, y, x + width, y + height])
            labels.append(self.category_mapping.get(ann['category_id'], -1))

        try:
            boxes = np.array(boxes, dtype=np.float32)
            labels = np.array(labels, dtype=np.int64)
        except ValueError as e:
            print(f"Invalid data for image ID {image_id}: {e}")
            return None, None

        # Normalize bounding boxes relative to the resized image
        image_height, image_width = image.shape[:2]
        boxes[:, [0, 2]] /= image_width  # Normalize x-min and x-max
        boxes[:, [1, 3]] /= image_height  # Normalize y-min and y-max

        # Apply augmentations
        if self.transform:
            transformed = self.transform(image=image, bboxes=boxes, labels=labels)
            image = transformed['image']
            boxes = transformed['bboxes']
            labels = transformed['labels']

        # Convert to PyTorch tensors
        if len(boxes) == 0 or len(labels) == 0:
            return None, None
        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)
        target = {"boxes": boxes, "labels": labels, "original_size": torch.tensor(original_size)}

        return image, target


# Load and parse JSON
def load_coco_json(json_path):
    try:
        with open(json_path, 'r') as f:
            coco_data = json.load(f)
        print("JSON file loaded successfully!")
        return coco_data
    except Exception as e:
        print(f"Error loading JSON file: {e}")
        return None


# Automatically detect GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


# Augmentation and Preprocessing
def get_augmentation_pipeline(mode='train'):
    if mode == 'train':
        return A.Compose([
            A.Resize(416, 416),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.2),  # New
            A.Rotate(limit=10, p=0.5),
            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),  # New
            A.RandomResizedCrop(416, 416, scale=(0.8, 1.0), ratio=(0.75, 1.33), p=0.5),  # New
            A.GaussianBlur(blur_limit=(3, 5), p=0.2),  # New
            A.GridDistortion(p=0.3),  # New
            A.Perspective(scale=(0.05, 0.1), p=0.3),  # New
            A.CoarseDropout(max_holes=8, max_height=32, max_width=32, fill_value=0, p=0.5),  # New
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))
    elif mode == 'val':
        return A.Compose([
            A.Resize(416, 416),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))


# Load data
json_path = '/content/drive/MyDrive/Colab Notebooks/instances_val2017.json'
coco_data = load_coco_json(json_path)
if coco_data:
    images = coco_data.get('images', [])
    annotations = coco_data.get('annotations', [])
    categories = coco_data.get('categories', [])
    category_mapping = {category['id']: idx for idx, category in enumerate(categories)}

    coco_dataset = CocoDataset(
        images,
        annotations,
        category_mapping,
        '/content/drive/MyDrive/Colab Notebooks/val2017',
        transform=get_augmentation_pipeline('train')
    )

    # Dynamic Batch Sizing
    memory_limit = torch.cuda.get_device_properties(0).total_memory // (1024 ** 3) if torch.cuda.is_available() else 8  # Default 8GB for CPU
    batch_size = min(max(4, memory_limit // 2), 16)  # Dynamic batch size between 4 and 16
    print(f"Using dynamic batch size: {batch_size}")

    # DataLoader
    num_workers = min(2, multiprocessing.cpu_count())
    data_loader = DataLoader(
        coco_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=lambda x: tuple(zip(*x)),
        num_workers=num_workers
    )

    # Process DataLoader with progress tracking
    valid_samples = 0
    skipped_samples = 0
    for batch in tqdm(data_loader, desc="Processing DataLoader"):
        images, targets = batch
        if images is not None and targets is not None:
            valid_samples += len(images)
            print(f"Processed batch: {len(images)} images")
        else:
            skipped_samples += 1

    print(f"Valid samples: {valid_samples}")
    print(f"Skipped samples: {skipped_samples}")
else:
    print("Failed to load COCO data.")


In [14]:
#improvement testing
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import json
import albumentations as A
from albumentations.pytorch import ToTensorV2
import multiprocessing
from tqdm import tqdm
import logging

# Initialize logging
logging.basicConfig(level=logging.INFO)

class CocoDataset(Dataset):
    def __init__(self, images, annotations, category_mapping, img_dir, transform=None, input_size=(416, 416)):
        self.images = images
        self.annotations = annotations
        self.category_mapping = category_mapping
        self.img_dir = img_dir
        self.transform = transform
        self.input_size = input_size
        self.image_id_to_annotations = self._group_annotations_by_image()
        logging.info(f"COCO Dataset initialized with {len(images)} images.")

    def _group_annotations_by_image(self):
        image_id_to_annotations = {}
        for ann in self.annotations:
            image_id = ann['image_id']
            if image_id not in image_id_to_annotations:
                image_id_to_annotations[image_id] = []
            image_id_to_annotations[image_id].append(ann)
        return image_id_to_annotations

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_info = self.images[idx]
        img_path = os.path.join(self.img_dir, image_info['file_name'])

        # Load and preprocess image
        try:
            image = cv2.imread(img_path)
            if image is None:
                raise FileNotFoundError(f"Image not found: {img_path}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            logging.error(f"Error loading image {img_path}: {e}")
            return None, None

        # Get original image size
        original_size = image.shape[:2]

        # Resize image dynamically to the input size
        image = cv2.resize(image, (self.input_size[1], self.input_size[0]))

        # Get annotations for the image
        image_id = image_info['id']
        annotations = self.image_id_to_annotations.get(image_id, [])
        if not annotations:
            logging.warning(f"No annotations found for image {img_path}")
            return None, None

        boxes = []
        labels = []
        for ann in annotations:
            try:
                x, y, width, height = ann['bbox']
                if width <= 0 or height <= 0:
                    raise ValueError(f"Invalid bbox dimensions: {ann['bbox']}")
                boxes.append([x, y, x + width, y + height])
                labels.append(self.category_mapping.get(ann['category_id'], -1))
            except KeyError as e:
                logging.error(f"Missing key in annotation for image {image_id}: {e}")
                continue
            except ValueError as e:
                logging.error(f"Invalid value in annotation for image {image_id}: {e}")
                continue

        # Normalize bounding boxes relative to the resized image
        try:
            boxes = np.array(boxes, dtype=np.float32)
            labels = np.array(labels, dtype=np.int64)
        except ValueError as e:
            logging.error(f"Invalid data for image ID {image_id}: {e}")
            return None, None

        image_height, image_width = image.shape[:2]
        boxes[:, [0, 2]] /= image_width  # Normalize x-min and x-max
        boxes[:, [1, 3]] /= image_height  # Normalize y-min and y-max

        # Apply augmentations
        if self.transform:
            transformed = self.transform(image=image, bboxes=boxes, labels=labels)
            image = transformed['image']
            boxes = transformed['bboxes']
            labels = transformed['labels']

        # Convert to PyTorch tensors
        if len(boxes) == 0 or len(labels) == 0:
            return None, None
        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)
        target = {"boxes": boxes, "labels": labels, "original_size": torch.tensor(original_size)}

        return image, target

# Load and parse JSON
def load_coco_json(json_path):
    try:
        with open(json_path, 'r') as f:
            coco_data = json.load(f)
        logging.info("JSON file loaded successfully!")
        return coco_data
    except Exception as e:
        logging.error(f"Error loading JSON file: {e}")
        return None


# Automatically detect GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f"Using device: {device}")


# Augmentation and Preprocessing
def get_augmentation_pipeline(mode='train'):
    if mode == 'train':
        return A.Compose([
            A.Resize(416, 416),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.2),
            A.Rotate(limit=10, p=0.5),
            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
            A.RandomResizedCrop(416, 416, scale=(0.8, 1.0), ratio=(0.75, 1.33), p=0.5),
            A.GaussianBlur(blur_limit=(3, 5), p=0.2),
            A.GridDistortion(p=0.3),
            A.Perspective(scale=(0.05, 0.1), p=0.3),
            A.CoarseDropout(max_holes=8, max_height=32, max_width=32, fill_value=0, p=0.5),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))
    elif mode == 'val':
        return A.Compose([
            A.Resize(416, 416),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

# Load data
json_path = '/content/drive/MyDrive/Colab Notebooks/instances_val2017.json'
coco_data = load_coco_json(json_path)
if coco_data:
    images = coco_data.get('images', [])
    annotations = coco_data.get('annotations', [])
    categories = coco_data.get('categories', [])
    category_mapping = {category['id']: idx for idx, category in enumerate(categories)}

    coco_dataset = CocoDataset(
        images,
        annotations,
        category_mapping,
        '/content/drive/MyDrive/Colab Notebooks/val2017',
        transform=get_augmentation_pipeline('train')
    )

    # Dynamic Batch Sizing
    if torch.cuda.is_available():
        memory_limit = torch.cuda.get_device_properties(0).total_memory * 0.8  # 80% of available GPU memory
        batch_size = 16  # Start with a default batch size
    else:
        memory_limit = 0  # Default to 0 for CPU
        batch_size = 4  # Use smaller batch size for CPU

    while batch_size > 1:
        try:
            data_loader = DataLoader(
                coco_dataset, batch_size=batch_size, shuffle=True,
                num_workers=multiprocessing.cpu_count(), collate_fn=lambda x: tuple(zip(*x))
            )
            break
        except RuntimeError as e:
            if 'out of memory' in str(e):
                logging.warning(f"Batch size {batch_size} too large, reducing.")
                batch_size -= 2
            else:
                logging.error(f"Error in data loader: {e}")
                break

    logging.info(f"Final batch size: {batch_size}")

    # A simple example of loading a batch of data (to verify everything is working)
    for images, targets in data_loader:
        if images is not None:
            logging.info(f"Loaded batch with {len(images)} images.")
        else:
            logging.warning("Empty batch loaded!")
        break  # Only load one batch for testing


/usr/local/lib/python3.10/dist-packages/albumentations/core/bbox_utils.py:478: RuntimeWarning: invalid value encountered in divide
  & (clipped_box_areas / denormalized_box_areas >= min_visibility - epsilon)
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/dropout/functional.py:211: RuntimeWarning: invalid value encountered in divide
  visibility_ratio = 1 - (intersection_area / box_areas[i])
/usr/local/lib/python3.10/dist-packages/albumentations/core/bbox_utils.py:478: RuntimeWarning: invalid value encountered in divide
  & (clipped_box_areas / denormalized_box_areas >= min_visibility - epsilon)
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/dropout/functional.py:211: RuntimeWarning: invalid value encountered in divide
  visibility_ratio = 1 - (intersection_area / box_areas[i])
